In [1]:
import os
import numpy as np
import pretty_midi
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import warnings

# Ignore specific warning from pretty_midi
warnings.filterwarnings('ignore', category=RuntimeWarning, module='pretty_midi.pretty_midi')

2023-08-13 18:28:31.974008: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import warnings

# Ignore specific warning from pretty_midi
warnings.filterwarnings('ignore', category=RuntimeWarning, module='pretty_midi.pretty_midi')

In [3]:
# Function to convert MIDI to piano roll
def midi_to_piano_roll(midi_file):
    try:
        midi = pretty_midi.PrettyMIDI(midi_file)
        piano_roll = midi.get_piano_roll(fs=25)
        return np.asarray(piano_roll)
    except Exception as e:
        print(f"Error in {midi_file}: {e}")
        return None

# Load the data
def load_data_from_folder(folder):
    composers = os.listdir(folder)
    labels, piano_rolls = [], []
    for composer in composers:
        composer_path = os.path.join(folder, composer)
        if os.path.isdir(composer_path):
            for midi_file in os.listdir(composer_path):
                if midi_file.endswith('.midi') or midi_file.endswith('.mid'):
                    piano_roll = midi_to_piano_roll(os.path.join(composer_path, midi_file))
                    if piano_roll is not None:
                        labels.append(composer)
                        piano_rolls.append(piano_roll)
    return piano_rolls, labels

In [4]:
train_data, train_labels = load_data_from_folder('/Users/fediajahangiry/Downloads/FP_AAI_511_NNL_SUM2_Team3/Composer_Dataset2/NN_midi_files_extended/train')
dev_data, dev_labels = load_data_from_folder('/Users/fediajahangiry/Downloads/FP_AAI_511_NNL_SUM2_Team3/Composer_Dataset2/NN_midi_files_extended/test')
test_data, test_labels = load_data_from_folder('/Users/fediajahangiry/Downloads/FP_AAI_511_NNL_SUM2_Team3/Composer_Dataset2/NN_midi_files_extended/dev')

In [5]:
# Encoding labels
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(train_labels)
y_train = to_categorical(y_train)

y_dev = label_encoder.transform(dev_labels)
y_dev = to_categorical(y_dev)

y_test = label_encoder.transform(test_labels)
y_test = to_categorical(y_test)

In [6]:
sample_shape = train_data[0].shape
print(sample_shape)

(128, 14016)


In [7]:
sample_shape = dev_data[0].shape
print(sample_shape)

(128, 10580)


In [8]:
sample_shape = test_data[0].shape
print(sample_shape)

(128, 8764)


In [9]:
fixed_length = 10627  # Chosen based on the 75th percentile

# Function to limit sequences to a fixed length
def limit_sequence_length(data, fixed_length):
    limited_data = []
    for sample in data:
        if sample.shape[1] > fixed_length:
            limited_sample = sample[:, :fixed_length]
        else:
            limited_sample = np.pad(sample, ((0, 0), (0, fixed_length - sample.shape[1])), mode='constant', constant_values=0)
        limited_data.append(limited_sample)
    return np.array(limited_data)

# Adjust all datasets
X_train = limit_sequence_length(train_data, fixed_length).reshape(-1, 128, fixed_length, 1)
X_dev = limit_sequence_length(dev_data, fixed_length).reshape(-1, 128, fixed_length, 1)
X_test = limit_sequence_length(test_data, fixed_length).reshape(-1, 128, fixed_length, 1)


In [ ]:
number_of_classes = y_train.shape[1]

# Define the CNN model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(128, 10627, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(number_of_classes, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, validation_data=(X_dev, y_dev), epochs=10, batch_size=32)

Epoch 1/10
 7/12 [================>.............] - ETA: 18:45 - loss: 1534.1906 - accuracy: 0.1786

In [ ]:
from tensorflow.keras.optimizers import RMSprop

model = Sequential()

# First Convolutional Layer
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(128, 128, 1)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

# Second Convolutional Layer
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten and Fully Connected Layers
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(y_train_categorical.shape[1], activation='softmax'))

# Use RMSprop optimizer as an alternative
optimizer = RMSprop(learning_rate=0.001)

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
model.fit(X_train, y_train_categorical, validation_data=(X_dev, y_dev_categorical), epochs=100, batch_size=32, callbacks=[early_stopping])

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.utils import to_categorical

In [ ]:
# Load CSV data
train_df = pd.read_csv('/Users/fediajahangiry/Downloads/FP_AAI_511_NNL_SUM2_Team3/csv_file/midi_train_processed.csv')
dev_df = pd.read_csv('/Users/fediajahangiry/Downloads/FP_AAI_511_NNL_SUM2_Team3/csv_file/midi_test_processed.csv')
test_df = pd.read_csv('/Users/fediajahangiry/Downloads/FP_AAI_511_NNL_SUM2_Team3/csv_file/midi_val_processed.csv')

In [ ]:
timesteps = 128
features_per_timestep = X_train.shape[1] // timesteps

In [ ]:
num_sequences = X_train.shape[1] // timesteps
print(num_sequences)

In [ ]:
features_per_timestep = X_train.shape[1] // num_sequences
print(features_per_timestep)

In [ ]:
# Extract features and labels
X_train = train_df.iloc[:, :-1].values
y_train = train_df.iloc[:, -1].values
X_dev = dev_df.iloc[:, :-1].values
y_dev = dev_df.iloc[:, -1].values
X_test = test_df.iloc[:, :-1].values
y_test = test_df.iloc[:, -1].values

# Define fixed values based on the provided info
timesteps = 128
num_sequences = 117
features_per_timestep = 128

# Pad or truncate sequences to a fixed length
def pad_or_truncate(data, fixed_length):
    padded_data = []
    for sample in data:
        if len(sample) < fixed_length:
            padded_data.append(np.pad(sample, (0, fixed_length - len(sample)), 'constant'))
        else:
            padded_data.append(sample[:fixed_length])
    return np.array(padded_data)

# Adjust the X_train, X_dev and X_test to match the expected length
expected_length = timesteps * features_per_timestep
X_train = pad_or_truncate(X_train, expected_length)
X_dev = pad_or_truncate(X_dev, expected_length)
X_test = pad_or_truncate(X_test, expected_length)

# Reshape data for CNN
X_train = X_train.reshape(-1, timesteps, features_per_timestep, 1)
X_dev = X_dev.reshape(-1, timesteps, features_per_timestep, 1)
X_test = X_test.reshape(-1, timesteps, features_per_timestep, 1)

# Encode labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_dev_encoded = label_encoder.transform(y_dev)
y_test_encoded = label_encoder.transform(y_test)

y_train_categorical = to_categorical(y_train_encoded)
y_dev_categorical = to_categorical(y_dev_encoded)
y_test_categorical = to_categorical(y_test_encoded)


In [ ]:
# Define the CNN model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(timesteps, features_per_timestep, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(y_train_categorical.shape[1], activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train_categorical, validation_data=(X_dev, y_dev_categorical), epochs=10, batch_size=32)

In [ ]:
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5)
model.fit(X_train, y_train_categorical, validation_data=(X_dev, y_dev_categorical), epochs=100, batch_size=32, callbacks=[early_stopping])


In [ ]:
from keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint('best_weights.h5', save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
from keras.callbacks import EarlyStopping


# This callback will monitor the validation loss (val_loss) of your model.
# If the validation loss doesn't improve for 5 (patience=5) consecutive epochs, it'll stop the training.
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

# fitting the model, include the callback in the 'callbacks' parameter.
# due to the early stopping callback, it may stop earlier if the validation loss doesn't improve for 5 consecutive epochs.
model.fit(X_train, y_train_categorical, validation_data=(X_dev, y_dev_categorical), epochs=100, batch_size=32, callbacks=[early_stopping])


early_stopping = EarlyStopping(monitor='val_loss', patience=100, restore_best_weights=True)
model.fit(X_train, y_train_categorical, validation_data=(X_dev, y_dev_categorical), epochs=100, batch_size=32, callbacks=[early_stopping])


In [ ]:
from keras.layers import Dropout, BatchNormalization
from keras.regularizers import l2

model = Sequential()

# First Convolutional Layer
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(128, 128, 1)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))  # Add dropout of 25%

# Second Convolutional Layer
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))  # Add dropout of 25%

# Flatten and Fully Connected Layers
model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.01)))  # L2 regularization
model.add(Dropout(0.5))  # Add dropout of 50%
model.add(Dense(y_train_categorical.shape[1], activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
model.fit(X_train, y_train_categorical, validation_data=(X_dev, y_dev_categorical), epochs=100, batch_size=32, callbacks=[early_stopping])


In [ ]:
from tensorflow.keras.optimizers import RMSprop

model = Sequential()

# First Convolutional Layer
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(128, 128, 1)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

# Second Convolutional Layer
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten and Fully Connected Layers
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(y_train_categorical.shape[1], activation='softmax'))

# Use RMSprop optimizer as an alternative
optimizer = RMSprop(learning_rate=0.001)

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
model.fit(X_train, y_train_categorical, validation_data=(X_dev, y_dev_categorical), epochs=100, batch_size=32, callbacks=[early_stopping])